In [1]:
%%capture
!pip install ht CoolProp fluids xlrd xlwt

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xlwt as xw
lista_data = []
import matplotlib.pyplot as plt
import ht,fluids,os,socket
import CoolProp as cp
from IPython.display import Latex
from prop_fluidos_intercambiadores import fluido_intercambiador



Determinar la superficie de intercambio de calor y el número de secciones necesarias de un intercambiador de calor de agua-agua del tipo tubos concéntricos (tubo metido en otro). El agua calefactora se mueve por el tubo interior de acero ($\lambda=$\SI{145}{\watt /\meter\degreeCelsius}) cuyo diámetro es $d_2/d_1=32$mm. La temperatura a la entrada del flujo másico de 2130kg/h que circula en ese circuito es de 95$^\circ$C 
El agua que se calienta circula en contracorriente por el canal anular entre los tubos y se calienta desde 15$^\circ$C hasta 45$^\circ$C. El diámetro interior del tubo es 48 mm. El flujo másico es de 3200 kg/h. Por razones de espacio la longitud máxima de una sección es de $1.9$ mts.
Como hipótesis se pueden ignorar las pérdidas de calor a través de la superficie externa del intercambiador.

## Intercambiador de Calor
#### HIPOTESIS DE CALCULO:


1. Intercambiador de calor a contracorriente de tubos concéntricos


In [2]:
#constantes de calculo
g = 9.8; cal2joule = 4.184
###########################################################
#           DATOS  del Intercambiador  
###########################################################
# Definición de Flujos
#Crudo = fluido_intercambiador('Crudo') 
#Glicol = fluido_intercambiador('Glycol')
# Temperaturas
T_agua1_e = 90 ##(al menos Twat + 20)


T_agua2_e = 15
T_agua2_s = 45
Caudal_agua1 = 2130*1/3600 #kg/s
Caudal_agua2 = 3200/3600 #kg/s



D_interno1 = 32e-3
D_interno2 = 35e-3
D_externo = 48e-3
espesor_tubo = (D_interno2-D_interno1)/2
Longitud = 1.9

# Propiedades Físicas
T_agua2_media = (T_agua1_e+T_agua2_e)/2
T_agua1_media = (T_agua1_e+T_agua2_e)/2  # en caso de faltar un temperatura, tomar de la otra corriente


Area_agua1 = (D_externo**2-(D_interno1+espesor_tubo)**2)*np.pi/4
Area_agua2 = D_interno1**2*np.pi/4

material = ht.nearest_material('steel')
k_acero = ht.k_material(material)

In [3]:

Cp_agua2 = cp.CoolProp.PropsSI('C','T',273+T_agua2_media,'P',101325,'Water')
Cp_agua1= cp.CoolProp.PropsSI('C','T',273+T_agua1_media,'P',101325,'Water')

rho_agua1 = cp.CoolProp.PropsSI('D','T',273+ T_agua1_media,'P',101325,'Water')
rho_agua2 = cp.CoolProp.PropsSI('D','T',273+ T_agua2_media,'P',101325,'Water')
k_agua1 = cp.CoolProp.PropsSI('L','T',273+ T_agua1_media,'P',101325,'Water')
mu_agua1 = cp.CoolProp.PropsSI('V','T',273+T_agua1_media,'P',101325,'Water')

k_agua2 = cp.CoolProp.PropsSI('L','T',273+ T_agua2_media,'P',101325,'Water')
mu_agua2 = cp.CoolProp.PropsSI('V','T',273+T_agua2_media,'P',101325,'Water')

nu_agua2 = mu_agua2/rho_agua2



nu_agua1 = mu_agua1 / rho_agua1

a_agua2 = k_agua2/(rho_agua2*Cp_agua2)    #difusividad térmica
a_agua1 = k_agua1/(rho_agua1*Cp_agua1)

Pr_agua1 = nu_agua1/a_agua1
Pr_agua2 = nu_agua2/a_agua2


In [4]:
Q2 = Caudal_agua2*Cp_agua2*(T_agua2_s-T_agua2_e)

Q1=Q2
T_agua1_s = T_agua1_e - Q1/Caudal_agua1/Cp_agua1

In [5]:
Veloc_agua1= Caudal_agua1/Area_agua1/rho_agua1
Veloc_agua2 = Caudal_agua2/Area_agua2/rho_agua2


Re_agua2 = Veloc_agua2 * D_interno1 / nu_agua2
D_agua1 = D_externo - D_interno1 - espesor_tubo
D_agua2 = D_interno1
Re_agua1 = Veloc_agua1 * D_agua1 / nu_agua1

Re_agua1,Re_agua2


#Nu_aceite = 3.66
Nu_agua2 = ht.Nu_conv_internal(Re_agua2,Pr_agua2)
Nu_agua1 = ht.Nu_conv_internal(Re_agua1,Pr_agua1)

h_agua2 = Nu_agua2 * k_agua2 / D_agua2
h_agua1 = Nu_agua1 * k_agua1 / D_agua1

In [59]:
D_agua1*1e3,k_agua1

(14.499999999999998, 0.6432122621860747)

In [56]:
Re_agua1,Re_agua2
Veloc_agua1,Veloc_agua2
print(Nu_agua1,Nu_agua2)

101.58659797027087 313.1478265168658


In [7]:


np.log((D_interno1+espesor_tubo)/D_interno1)/k_acero 


1/ (h_agua1*D_interno1)

0.006934693841037895

In [17]:
h_agua2,h_agua1

(6294.391309142678, 4506.327275051396)

In [23]:
k_acero = ht.k_material('stainless steel')
#espesor_tubo = 1e-3

Ulimpio_d = (1/ (h_agua2*D_interno1) + np.log((D_interno1+2*espesor_tubo)/D_interno1)/k_acero + 1/(h_agua1*(D_agua1)))**-1
#U_calc = 1 / (1/Ulimpio + Rf *(1+Tubo_dext/Tubo_dint))
Ulimpio = Ulimpio_d /D_interno1
display(Latex(f'$U={Ulimpio:.1f}$ watt/K'))

<IPython.core.display.Latex object>

[datos](https://material-properties.org/es/combustible-diesel-densidad-punto-de-fusion-conductividad-termica/)

In [24]:
Cmin = np.min([Cp_agua1*Caudal_agua1,Cp_agua2*Caudal_agua2])
Cmax = np.max([Cp_agua1*Caudal_agua1,Cp_agua2*Caudal_agua2])

In [25]:
Cr = Cmin/Cmax

In [26]:
Qmax = Cmin*(T_agua1_e-T_agua2_e)

In [27]:
epsilon = Q1/Qmax
print(epsilon)

0.6009389671361501


In [28]:
Nut = ht.NTU_from_effectiveness(epsilon,Cr)

In [29]:
A = Nut*Cmin/Ulimpio

In [30]:
A

2.4664389545739787

In [37]:
Area_tubo = D_externo*np.pi*Longitud
Npasos = A/Area_tubo
Npasos = np.ceil(Npasos)
A = Npasos * Area_tubo
UA = Ulimpio*A
Nut_r = UA/Cmin

epsilon_r = ht.effectiveness_from_NTU(Nut_r,Cr)

In [38]:
epsilon_r

0.6139092817416095

El caudal de agua fria se reduce en un 15%.

In [43]:
Caudal_agua2b = Caudal_agua2*0.85
Cfb = Cp_agua2*Caudal_agua2b
Crb = Cmin/Cfb
epsilon_b = ht.effectiveness_from_NTU(Nut_r,Crb)

In [45]:
Qb = Qmax*epsilon_b

In [62]:
Qb,epsilon_b,epsilon

(110411.80919806474, 0.5949599632873669, 0.6009389671361501)

In [48]:
Q1

111521.38408189183

In [49]:
Qmax

185578.55319877315

In [50]:
Nut

1.2196344985513807

In [51]:
h_agua1,h_agua2

(4506.327275051396, 6294.391309142678)

In [53]:
Ulimpio

1223.5616332755667

In [54]:
Area_tubo

0.28651325000738914

In [55]:
Npasos

9.0